In [10]:
import sqlite3

from collections import Counter

import pandas as pd

In [11]:
sql = """
    SELECT
        I.id AS pair_id,
        PA.name AS a_project,
        FA.filename AS a_name,
        FA.content AS a_content,
        PB.name AS b_project,
        FB.filename AS b_name,
        FB.content AS b_content,
        U.username,
        R.value
    FROM fileranker_response R
    JOIN auth_user U ON U.id = R.user_id
    JOIN fileranker_sequenceitem I ON I.id = R.item_id
    JOIN fileranker_sequence S ON S.id = I.sequence_id
    JOIN fileranker_file FA ON FA.id = I.file_a_id
    JOIN fileranker_file FB ON FB.id = I.file_b_id
    JOIN fileranker_project PA ON PA.id = FA.project_id
    JOIN fileranker_project PB ON PB.id = FB.project_id
    WHERE S.name = 'survey'
    ORDER BY I.position, U.id
"""

res_df = pd.read_sql(sql=sql, con=sqlite3.connect("results.sqlite3"))
res_df = res_df[~(res_df["username"] == "admin")]
res_df.insert(3, "a_loc", [len(c.split("\n")) for c in res_df["a_content"]])
res_df.insert(5, "b_loc", [len(c.split("\n")) for c in res_df["b_content"]])
res_df = res_df.drop(["a_content", "b_content"], axis=1)
res_df

,pair_id,a_project,a_name,a_loc,b_loc,b_project,b_name,username,value
1,1,survey5,src/main/java/LinearScale.java,33,37,survey5,src/main/java/Main.java,chipmunk,A
2,1,survey5,src/main/java/LinearScale.java,33,37,survey5,src/main/java/Main.java,jackal,U
3,1,survey5,src/main/java/LinearScale.java,33,37,survey5,src/main/java/Main.java,wildcat,B
5,2,survey1,src/com/surveysystem/question/ShortAnswer.java,28,17,survey2,src/answers/Tabulator.java,chipmunk,B
6,2,survey1,src/com/surveysystem/question/ShortAnswer.java,28,17,survey2,src/answers/Tabulator.java,jackal,B
...,...,...,...,...,...,...,...,...,...
161,50,survey2,src/questionnaires/Survey.java,162,212,survey3,src/Survey.java,jackal,A
162,50,survey2,src/questionnaires/Survey.java,162,212,survey3,src/Survey.java,wildcat,U
163,51,survey5,src/main/java/AnswerSheet.java,54,71,survey6,src/Question/Question.java,chipmunk,U
164,51,survey5,src/main/java/AnswerSheet.java,54,71,survey6,src/Question/Question.java,jackal,A


In [12]:
res_df.to_csv("reports/votes_raw.csv")

In [13]:
consensus_rows = []
groups = list(res_df.groupby(["pair_id", "a_project", "a_name", "b_project", "b_name"]))

total_agree = 0

for (pair_id, a_proj, a_name, b_proj, b_name), group_df in groups:
    values = list(group_df["value"])
    if len(values) != 3:
        continue
    if len(set(values)) == 1:
        total_agree += 1
    counts = Counter(values)
    if counts["A"] >= 2 and counts["B"] == 0:
        consensus_rows.append({
            "p_proj": a_proj,
            "p_name": a_name,
            "n_proj": b_proj,
            "n_name": b_name,
        })
    elif counts["B"] >= 2 and counts["A"] == 0:
        consensus_rows.append({
            "p_proj": b_proj,
            "p_name": b_name,
            "n_proj": a_proj,
            "n_name": a_name,
        })

print(total_agree)
consensus_df = pd.DataFrame.from_records(consensus_rows)
consensus_df

17


,p_proj,p_name,n_proj,n_name
0,survey2,src/answers/Tabulator.java,survey1,src/com/surveysystem/question/ShortAnswer.java
1,survey1,src/com/surveysystem/util/Output.java,survey6,src/Menu/MenuOptions/LoadOption.java
2,survey1,src/com/surveysystem/question/ShortAnswer.java,survey6,src/Calculating/Grader.java
3,survey6,src/Menu/SurveyMenu.java,survey6,src/Import/SerializerImport.java
4,survey3,src/CsvExportFormat.java,survey2,src/ui/GUI.java
5,survey6,src/Export/JsonExport.java,survey6,src/Question/EssayQuestion.java
6,survey6,src/Survey/Survey.java,survey5,src/main/java/SurveyTakerGUI.java
7,survey6,src/Menu/TestMenu.java,survey3,src/ValidDate.java
8,survey6,src/InputHandler/InputHandlerFactory.java,survey1,src/com/surveysystem/io/SerializedOutput.java
9,survey3,src/Question.java,survey5,src/main/java/AnswerSheet.java


In [21]:
len(consensus_df)

33

In [14]:
def to_team_id(db_name: str) -> int:
    return int(db_name.split("/")[-1].split("_")[0][-1])

lcom_df = pd.read_excel("reports/report_survey.xlsx")
lcom_df.insert(0, "project", [f"survey{to_team_id(d)}" for d in lcom_df["DB"]])
lcom_df = lcom_df.drop(["DB", "Name", "Kind"], axis=1)

def get_score(project, filename, score_name):
    df = lcom_df[(lcom_df["project"] == project) & (lcom_df["Filename"] == filename)]
    if len(df) != 1:
        raise RuntimeError
    return df.iloc[0][score_name]

lcom_df

,project,Filename,[E] LOC,[E] Entities,[E] Fields,[E] Methods,[C] LCOM1,[C] LCOM2,[C] LCOM3,[C] MSC1,[C] MSC2,[C] MSC3,[C] MSC4
0,survey1,src/YamlLoader.java,28,2,1,1,0,0.000000,NaN,1.467331,1.467336,4.306150,4.306150
1,survey1,src/com/surveysystem/form/Survey.java,45,10,3,7,15,0.714286,0.833333,2.370305,3.228553,6.431171,3.108965
2,survey1,src/com/surveysystem/io/CSVInput.java,33,4,0,4,6,NaN,NaN,2.256124,2.691698,6.880228,3.375875
3,survey1,src/com/surveysystem/io/CSVOutput.java,70,3,0,3,3,NaN,NaN,2.255224,2.541165,7.724497,5.053195
4,survey1,src/com/surveysystem/io/InputStrategy.java,18,3,0,3,3,NaN,NaN,2.068788,2.373027,6.494263,4.251490
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,survey7,src/main/java/TakenSurvey.java,22,4,1,3,3,0.666667,1.000000,2.278257,2.523849,7.072925,3.874886
181,survey7,src/main/java/TestPanel.java,116,3,2,1,0,0.000000,NaN,2.256038,2.543164,7.716712,4.979432
182,survey7,src/main/java/TestResponse.java,54,15,6,9,33,0.870370,0.979167,2.728239,3.432583,8.135342,2.646065
183,survey7,src/main/java/TrueFalseQuestion.java,37,7,2,5,10,1.000000,1.250000,2.586631,3.204678,7.985504,2.823066


In [15]:
consensus_df["p_loc"] = [get_score(r["p_proj"], r["p_name"], "[E] LOC") for _, r in consensus_df.iterrows()]
consensus_df["p_lcom1"] = [get_score(r["p_proj"], r["p_name"], "[C] LCOM1") for _, r in consensus_df.iterrows()]
consensus_df["p_lcom2"] = [get_score(r["p_proj"], r["p_name"], "[C] LCOM2") for _, r in consensus_df.iterrows()]
consensus_df["p_lcom3"] = [get_score(r["p_proj"], r["p_name"], "[C] LCOM3") for _, r in consensus_df.iterrows()]
consensus_df["p_msc1"] = [get_score(r["p_proj"], r["p_name"], "[C] MSC1") for _, r in consensus_df.iterrows()]
consensus_df["p_msc2"] = [get_score(r["p_proj"], r["p_name"], "[C] MSC2") for _, r in consensus_df.iterrows()]

consensus_df["n_loc"] = [get_score(r["n_proj"], r["n_name"], "[E] LOC") for _, r in consensus_df.iterrows()]
consensus_df["n_lcom1"] = [get_score(r["n_proj"], r["n_name"], "[C] LCOM1") for _, r in consensus_df.iterrows()]
consensus_df["n_lcom2"] = [get_score(r["n_proj"], r["n_name"], "[C] LCOM2") for _, r in consensus_df.iterrows()]
consensus_df["n_lcom3"] = [get_score(r["n_proj"], r["n_name"], "[C] LCOM3") for _, r in consensus_df.iterrows()]
consensus_df["n_msc1"] = [get_score(r["n_proj"], r["n_name"], "[C] MSC1") for _, r in consensus_df.iterrows()]
consensus_df["n_msc2"] = [get_score(r["n_proj"], r["n_name"], "[C] MSC2") for _, r in consensus_df.iterrows()]

In [16]:
consensus_df["loc"] = consensus_df["p_loc"] < consensus_df["n_loc"]
consensus_df["lcom1"] = consensus_df["p_lcom1"] < consensus_df["n_lcom1"]
consensus_df["lcom2"] = consensus_df["p_lcom2"] < consensus_df["n_lcom2"]
consensus_df["lcom3"] = consensus_df["p_lcom3"] < consensus_df["n_lcom3"]
consensus_df["msc1"] = consensus_df["p_msc1"] < consensus_df["n_msc1"]
consensus_df["msc2"] = consensus_df["p_msc2"] < consensus_df["n_msc2"]

In [17]:
consensus_df.to_csv("reports/votes_consensus.csv")

In [18]:
print("LOC: {}".format(consensus_df["loc"].sum()))
print("LCOM1: {}".format(consensus_df["lcom1"].sum()))
print("LCOM2: {}".format(consensus_df["lcom2"].sum()))
print("LCOM3: {}".format(consensus_df["lcom3"].sum()))
print("MSC1: {}".format(consensus_df["msc1"].sum()))
print("MSC2: {}".format(consensus_df["msc2"].sum()))

LOC: 29
LCOM1: 26
LCOM2: 12
LCOM3: 11
MSC1: 24
MSC2: 26


In [20]:
0.14 * 24

3.3600000000000003